In [1]:
from openai import OpenAI
import os
import re
import pandas as pd
from dotenv import load_dotenv
import requests

In [2]:
# Parent directory of the current directory
path = os.path.dirname(os.getcwd()) 

output_folder = path+'/results/predictions'
zero_simple_path = os.path.join(output_folder, 'zero-shot_simple-prompt')

In [3]:
load_dotenv()

True

In [4]:
openai_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=openai_API_KEY)

In [5]:
claude_t1 = open(zero_simple_path+'/claude-3-5-sonnet-20240620/transcription_1.txt', 'r', encoding='utf-8').read()
claude_t1

'Here\'s a recreation of the content of the table in the image:\n\n| CLASSE DES DÉCRETS | DÉSIGNATION DES PERSONNES DÉCÉDÉES OU ABSENTES | NOMS, PRÉNOMS | QUALITÉ | DOMICILE | DATE DE NAISSANCE | LIEU DE NAISSANCE | OBSERVATIONS |\n|-------------------|------------------------------------------------|---------------|---------|----------|-------------------|-------------------|---------------|\n| 398 Trente huit | Hoorrent | Alphonse | Ouvrier (1849) | Roubaix (Nord) | 22/8/1825 | 1825 | Arrêté du vingt huit octobre 1879 (Absent) |\n| 399 " | Lefebvre | Jules | Ouvrier 6 octobre 1879 | Roubaix (Nord) | 12/5/ | 1823 | Arrêté du trente octobre 1879 (Absent) |\n| 399 bis " | Desmedt | Gustave | Nicollas 18 mars | Courtrai (Belgique) | 9/10 | (30) 49,50 | Arrêté du deux novembre 1879 (Absent) |\n| 400 " | Monseur | Pascal Henri | Célibataire 4 Août 41 | Roubaix-Tourcoing | 29/3/40 | 1840 45 | 55 " " |\n| 401 " | Bouty | Fidèle | Ouvrier 6 octobre 1879 | Roubaix (Nord) | 25/9/ | 25/9/ | 15 "

In [22]:
def call(file):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai_API_KEY}"
    }

    payload = {
        "model": "gpt-4o",
        "messages":  [
                    {
                        "role": "user",
                        "content": [
                        {
                            "type": "text",
                            "text": f"""
                            This is an output from you. Clean it such that we only have the table without any separators: {file}
                            """
                        }
                        ]
                    }

                    ],
        "max_tokens": 1000,
        "temperature": 0
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    content = response.json()["choices"][0]["message"]["content"]
    return content

In [24]:
claude_t1_cleaned = call(claude_t1)
print(claude_t1_cleaned)

CLASSE DES DÉCRETS DÉSIGNATION DES PERSONNES DÉCÉDÉES OU ABSENTES NOMS, PRÉNOMS QUALITÉ DOMICILE DATE DE NAISSANCE LIEU DE NAISSANCE OBSERVATIONS
398 Trente huit Hoorrent Alphonse Ouvrier (1849) Roubaix (Nord) 22/8/1825 1825 Arrêté du vingt huit octobre 1879 (Absent)
399 " Lefebvre Jules Ouvrier 6 octobre 1879 Roubaix (Nord) 12/5/ 1823 Arrêté du trente octobre 1879 (Absent)
399 bis " Desmedt Gustave Nicollas 18 mars Courtrai (Belgique) 9/10 (30) 49,50 Arrêté du deux novembre 1879 (Absent)
400 " Monseur Pascal Henri Célibataire 4 Août 41 Roubaix-Tourcoing 29/3/40 1840 45 55 " "
401 " Bouty Fidèle Ouvrier 6 octobre 1879 Roubaix (Nord) 25/9/ 25/9/ 15 " 22 Juillet
402 quatre cent deux Godart Constable Ouvrier 22 mars Roubaix (Nord) 15/5/1 1779/1 30 "


In [32]:
gpt4o_t1 = open(zero_simple_path+'/gpt-4o/transcription_1.txt', 'r', encoding='utf-8').read()
print(gpt4o_t1)

| DATE DE DÉCÈS | NOM | PRÉNOM | DOMICILE | PROFESSION | DATE DE NAISSANCE | LIEU DE NAISSANCE | DATE DE DÉPÔT | NATURE DU DÉPÔT | DATE DE L'ACTE DE DÉCÈS | DATE DE L'ACTE DE NAISSANCE | OBSERVATIONS |
|---------------|-----|--------|-----------|-------------|----------------|----------------|--------------|----------------|----------------------|----------------------|--------------|
| 6 1923 (mois illisible) | Heverent | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) |
| 6 1923 (mois illisible) | Lefevre | Jules | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) |
| 6 1923 (mois illisible) | Desmedt | Jeanne | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) | (illisible) |
| 6 1923 (mois illisible) | Monseur | (illisible) | (illisible) | (illisible) | (illisib

In [30]:
gpt4o_t1_cleaned = call(gpt4o_t1)
print(gpt4o_t1_cleaned )

DATE DE DÉCÈS NOM PRÉNOM DOMICILE PROFESSION DATE DE NAISSANCE LIEU DE NAISSANCE DATE DE DÉPÔT NATURE DU DÉPÔT DATE DE L'ACTE DE DÉCÈS DATE DE L'ACTE DE NAISSANCE OBSERVATIONS
6 1923 (mois illisible) Heverent (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible)
6 1923 (mois illisible) Lefevre Jules (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible)
6 1923 (mois illisible) Desmedt Jeanne (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible)
6 1923 (mois illisible) Monseur (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible)
6 1923 (mois illisible) Bouly (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible) (illisible)
6 1923 (mois illisible) Godart (illisible) (illisibl